# Matrix Factorization
* Prediction is $\tilde R = UA^T$ 
* Loss fuction is $L = \lVert (R - \tilde R)^\Omega \rVert _2^2 + \lambda_u \lVert U \rVert _2^2 + \lambda_a \lVert A \rVert _2^2$
* $\Omega$ is the set of oberved pairs $(i, j)$
* $M^\Omega$ is the projection of $M$ onto $\Omega$ for any matrix $M$
* $U$ is an $m x k$ matrix, $A$ is an $n x k$ matrix and $R$ is the $m x n$ ratings matrix

# TODO residualize vs biases

In [1]:
using CSV
using DataFrames
using FileIO
using JLD2
using JupyterFormatter
using LinearAlgebra
using Optim
using ProgressMeter
using SparseArrays
using Statistics
import Metrics

In [2]:
enable_autoformat();

In [3]:
function get_split(split)
    @assert split in ["training", "validation"]
    file = "../../data/splits/$(split).csv"
    df = DataFrame(CSV.File(file))
    df.username .+= 1 # julia is 1 indexed
    df.anime_id .+= 1
    df.my_score = float(df.my_score)
    return df
end;

In [4]:
function write_prediction(df, split)
    @assert split in ["validation"]
    outdir = "../../data/alphas/$name"
    if !isdir(outdir)
        mkpath(outdir)
    end
    df = copy(df)
    df.username .-= 1
    df.anime_id .-= 1
    CSV.write("$(outdir)/$(split).csv", df)
end;

In [5]:
function write_model(params)
    outdir = "../../data/alphas/$name"
    if !isdir(outdir)
        mkpath(outdir)
    end
    save("$(outdir)/model.jld2", params)
end;

In [6]:
function evaluate(truth, pred)
    print("RMSE ", sqrt(Metrics.mse(pred, truth)))
    print(" MAE ", Metrics.mae(pred, truth))
    print(" R2 ", Metrics.r2_score(pred, truth))
end;

In [7]:
name = "MatrixFactorization";

In [8]:
training = get_split("training");

In [9]:
validation = get_split("validation");

# Alternating Least Squares Algorithm
* $u_{ik} = \dfrac{\sum_{j \in \Omega_i}(r_{ij} - \tilde r_{ij} + u_{ik}a_{kj})}{\sum_{j \in \Omega_i} a_j^2 + \lambda_u}$
* $\Omega$ is the set of (user, item) pairs that we have ratings for
* $\Omega_i$ is subset of $\Omega$ for which the user is the $i$-th user

In [10]:
function make_prediction(usernames, anime_ids, U, A)
    r = zeros(eltype(U), length(usernames))
    @showprogress for i = 1:length(r)
        if (usernames[i] <= size(U)[1]) && (anime_ids[i] <= size(A)[1])
            r[i] = dot(U[usernames[i], :], A[anime_ids[i], :])
        end
    end
    return r
end

make_prediction (generic function with 1 method)

In [11]:
function update_users!(usernames, anime_ids, ratings, U, A, λ_u)
    residual_ratings = ratings .- make_prediction(usernames, anime_ids, U, A)
    function make_sparse(data)
        return sparse(usernames, anime_ids, data, size(U)[1], size(A)[1])
    end
    residuals = make_sparse(residual_ratings)
    mask = make_sparse(fill(1.0, length(ratings)))

    @showprogress for i = 1:size(U)[1]
        a = A[:, :] .* mask[i, :]
        res_i = residuals[i, :] .+ (U[i, :]' .* a)
        U[i, :] = sum(res_i .* a, dims = 1) ./ (sum(a .* a, dims = 1) .+ λ_u)
    end
end

function update_items!(users, items, ratings, u, a, λ_a)
    update_user_biases!(items, users, ratings, a, u, λ_a)
end;

In [12]:
function train_model(training, λ_u, λ_a, K, ϵ = 1e-6)
    users = training.username
    items = training.anime_id
    ratings = training.my_score
    U = zeros(eltype(λ_u), maximum(users), K)
    A = zeros(eltype(λ_a), maximum(items), K)
    U .+= rand(size(U)...)
    A .+= rand(size(A)...)


    converged = false
    while !converged
        old_U = copy(U)
        old_A = copy(A)
        update_users!(users, items, ratings, U, A, λ_u)
        update_items!(users, items, ratings, U, A, λ_a)

        converged = (maximum(abs.(U - old_U)) < ϵ) && (maximum(abs.(A - old_A)) < ϵ)
        if converged
            break
        end
    end
    return U, A
end;

In [13]:
K = 1;
function validation_mse(λ)
    U, A = train_model(training, λ[1], λ[2], K)
    pred_score = make_prediction(validation.username, validation.anime_id, U, A)
    return Metrics.mse(validation.my_score, pred_score)
end;

## Training

In [ ]:
# Find the best regularization hyperparameters
res = optimize(
    validation_mse,
    fill(0.0, 2),
    fill(Inf, 2),
    fill(1.0, 2),
    Fminbox(LBFGS()),
    autodiff = :forward,
    Optim.Options(show_trace = true),
);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:16
Progress:   0%|▏                                        |  ETA: 0:47:29

In [ ]:
print("The optimal [λ_u, λ_a] is ", Optim.minimizer(res));

In [ ]:
U, A = train_model(training, Optim.minimizer(res)..., K);
model(users, items) = make_prediction(users, items, U, A);

## Inference

In [ ]:
training_pred_score = model(training.username, training.anime_id);
evaluate(training.my_score, training_pred_score);

In [ ]:
val_pred_score = model(validation.username, validation.anime_id);
evaluate(validation.my_score, val_pred_score);

In [ ]:
# write predictions to disk
val_pred = copy(validation);
val_pred.my_score = val_pred_score;
write_prediction(val_pred, "validation");

In [ ]:
# write model to disk
write_model(Dict("U" => U, "A" => A, "λ" => Optim.minimizer(res), "model" => model));